# Practica 2 - Natural Language processing

In [1]:
import os
import pandas as pd

data_dir = "data"
train_path = os.path.join(data_dir, "train.csv")
test_path = os.path.join(data_dir, "test.csv")
df_train = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
df_test = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
df_train.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [2]:
df_train.isna().sum()

,0
text,1
selected_text,1
sentiment,0
Time of Tweet,0
Age of User,0
Country,0
Population -2020,0
Land Area (Km²),0
Density (P/Km²),0


In [3]:
df_test.isna().sum()

,0
text,1281
sentiment,1281
Time of Tweet,1281
Age of User,1281
Country,1281
Population -2020,1281
Land Area (Km²),1281
Density (P/Km²),1281


In [4]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [5]:
df_train["text"].isna().sum()

np.int64(0)

In [7]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00


In [8]:
from tensorflow.keras.layers import TextVectorization

vectorize_layer = TextVectorization(
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

corpus = df_train["text"].values

vectorize_layer.adapt(corpus)

vectorized_train = vectorize_layer(corpus)
vectorized_train

<tf.Tensor: shape=(27480, 33), dtype=int64, numpy=
array([[  293,    17, 15185, ...,     0,     0,     0],
       [  413,   115,     2, ...,     0,     0,     0],
       [    6,  1335,    10, ...,     0,     0,     0],
       ...,
       [  225,    31,    12, ...,     0,     0,     0],
       [   20,     9,    28, ...,     0,     0,     0],
       [   29,    30,  6480, ...,     0,     0,     0]])>

In [9]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('i'),
 np.str_('to'),
 np.str_('the'),
 np.str_('a'),
 np.str_('my'),
 np.str_('and'),
 np.str_('you'),
 np.str_('it')]

In [10]:
import tensorflow as tf
window_size = 2
def generate_skipgram_pairs(sequence):
    # sequence: shape (sentence_length,)
    seq_len = tf.shape(sequence)[0]

    # Get all valid center word positions
    i = tf.range(seq_len)

    def get_context_pairs(i_t):
        start = tf.maximum(0, i_t - window_size)
        end = tf.minimum(seq_len, i_t + window_size + 1)

        center = sequence[i_t]
        # context = sequence[start:i_t] + sequence[i_t+1:end]
        left_context = sequence[start:i_t]
        right_context = sequence[i_t + 1:end]
        context = tf.concat([left_context, right_context], axis=0)

        center_tiled = tf.fill([tf.shape(context)[0]], center)

        return tf.data.Dataset.from_tensor_slices((center_tiled, context))

    return tf.data.Dataset.from_tensor_slices(i).flat_map(get_context_pairs)


In [14]:
train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)

vocab_size = len(vectorize_layer.get_vocabulary())
window_size = 3

print(train_ds.element_spec)
train_ds.as_numpy_iterator().next()

TensorSpec(shape=(33,), dtype=tf.int64, name=None)


array([  293,    17, 15185,    69,     2,   120,    47,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0])

In [12]:
train_ds = train_ds.flat_map(generate_skipgram_pairs)
print(train_ds.element_spec)
train_ds.as_numpy_iterator().next()

(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


(np.int64(293), np.int64(17))

In [15]:
train_ds = (
    train_ds
    .map(
        lambda x: ((tf.one_hot(x[0], depth=vocab_size), tf.one_hot(x[1], depth=vocab_size)), tf.constant(1.0)),
        num_parallel_calls=tf.data.AUTOTUNE,
        )
)
print(train_ds.element_spec)
next(train_ds.as_numpy_iterator())

((TensorSpec(shape=(29164,), dtype=tf.float32, name=None), TensorSpec(shape=(29164,), dtype=tf.float32, name=None)), TensorSpec(shape=(), dtype=tf.float32, name=None))


((array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
  array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)),
 np.float32(1.0))

In [16]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dot, Activation

# vocab_size = 29164
embedding_dim = 128

# One-hot inputs (shape = (vocab_size,))
input_target = Input(shape=(vocab_size,))
input_context = Input(shape=(vocab_size,))

# Shared embedding layer simulated with Dense layer
embedding_layer = Dense(embedding_dim, use_bias=False)

target_embedding = embedding_layer(input_target)   # shape: (embedding_dim,)
context_embedding = embedding_layer(input_context) # same

# Dot product of embeddings
dot_product = Dot(axes=-1)([target_embedding, context_embedding])

# Sigmoid output for skipgram-style binary prediction
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy')


In [17]:
model.fit(train_ds.batch(128).prefetch(tf.data.AUTOTUNE), epochs=5)


Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 45s 208ms/step - loss: 0.6476
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 43s 201ms/step - loss: 0.3082
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 43s 202ms/step - loss: 0.1743
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 43s 197ms/step - loss: 0.1097
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 43s 198ms/step - loss: 0.0724


In [18]:
embeddings = embedding_layer.get_weights()[0]

print(embeddings.shape)

(29164, 128)


In [21]:
# prompt: given vectorize_layer how to obtain the index for the word "what"

import numpy as np

# Get the vocabulary from the vectorize_layer
vocabulary = vectorize_layer.get_vocabulary()

# Find the index of the word "what"
try:
    what_index = vocabulary.index("what")
    print(f"The index of 'what' in the vocabulary is: {what_index}")
except ValueError:
    print("'what' is not found in the vocabulary.")


The index of 'what' in the vocabulary is: 57


In [22]:
embeddings[what_index]

array([-0.297594  , -0.18889844,  0.33601463,  0.2749642 , -0.3028369 ,
        0.29447892, -0.33340815,  0.32808417, -0.28593287,  0.3414568 ,
        0.30592406,  0.311685  ,  0.3015567 , -0.27318442, -0.17361256,
       -0.31516472,  0.3016468 , -0.32185522, -0.3343558 ,  0.30215746,
        0.34733737, -0.2756483 ,  0.3109338 ,  0.34284166,  0.3247285 ,
        0.3130445 ,  0.32104582, -0.2827067 , -0.32995856,  0.31195074,
       -0.3281337 , -0.24215686,  0.30152658,  0.333146  , -0.31881943,
        0.30683988,  0.28844053, -0.29022932, -0.29020125,  0.24783881,
        0.30345857,  0.31272623,  0.3257795 , -0.3265034 ,  0.09420276,
       -0.25440598, -0.2862201 , -0.34170637, -0.19805843,  0.3180254 ,
        0.32141227,  0.32560623,  0.26768762,  0.10990416, -0.31688923,
        0.1739782 , -0.30963588,  0.1181053 ,  0.3279269 , -0.21950777,
        0.20211726,  0.30731416, -0.33384553,  0.35427818, -0.31358153,
        0.31978387,  0.3430791 ,  0.3000202 ,  0.32862422, -0.31